In [6]:
!pip install piazza-api
!pip install html2text
!pip install pandas
!pip install openpyxl


  Using cached openpyxl-3.1.2-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached et_xmlfile-1.1.0-py3-none-any.whl.metadata (1.8 kB)
Using cached openpyxl-3.1.2-py2.py3-none-any.whl (249 kB)
Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)


In [8]:
import json
import re
import os
import time
from html2text import HTML2Text
from piazza_api import Piazza
import pandas as pd
from datetime import datetime

HTML_2_TEXT = HTML2Text()
HTML_2_TEXT.ignore_links = True

EMAIL = None
PASSWORD = None
NETWORK_ID = None

if os.path.exists('credentials.json'):
    with open('credentials.json') as f:
        credentials = json.load(f)
        EMAIL = credentials.get('email', None)
        PASSWORD = credentials.get('password', None)
        NETWORK_ID = credentials.get('network_id', None)

def count_good_comments(comment, student_comment_counts, user_id_to_name):
    if (comment['type'] == 'followup' or comment['type'] == 'feedback') and comment['tag_good']:
        student_id = comment.get('uid', None)
        if student_id and student_id not in student_comment_counts:
            student_comment_counts[student_id] = 0
        if student_id:
            student_comment_counts[student_id] += 1
            if student_id not in user_id_to_name:
                user_id_to_name[student_id] = "Unknown"
    for child in comment['children']:
        count_good_comments(child, student_comment_counts, user_id_to_name)

def main():
    """
    This function performs the main logic of the program.
    It logs in to Piazza, retrieves posts and comments, and exports the data to an Excel file.
    """

    piazza = Piazza()
    piazza.user_login(email=EMAIL, password=PASSWORD)
    print("Logged in successfully.")
    network_id = NETWORK_ID
    course = piazza.network(network_id)
    posts = list(course.iter_all_posts(limit=500))  # Convert generator to list
    print(posts)
    print(f"Found {len(posts)} posts.")

    # Get all users and create a dictionary mapping user IDs to names and emails
    all_users = course.get_all_users()
    user_id_to_name = {user['id']: user['name'] for user in all_users}
    user_id_to_email = {user['id']: user['email'] for user in all_users}

    student_comment_counts = {}

    for post in posts:
        for child in post['children']:
            count_good_comments(child, student_comment_counts, user_id_to_name)
        # time.sleep(1)  # Add a delay between requests

    # Prepare data for DataFrame
    data = []
    for uid, count in student_comment_counts.items():
        student_name = user_id_to_name.get(uid, "Unknown")
        student_email = user_id_to_email.get(uid, "Unknown")
        student_id = student_email.split('@')[0] if student_email != "Unknown" else "Unknown"
        data.append([student_id, student_name, count])
        print(f"Student {student_name} ({uid}) with ID {student_id} has {count} helpful comments.")

    # Create DataFrame and export to Excel
    df = pd.DataFrame(data, columns=['ID', 'Họ Tên', 'Số câu trả lời đúng'])
    df.to_excel(f"{datetime.now().strftime('%Y%m%d%H%M%S')}.xlsx", index=False)

if __name__ == '__main__':
    main()

Logged in successfully.
[{'history_size': 1, 'folders': [], 'nr': 5, 'data': {'embed_links': []}, 'created': '2024-02-20T09:28:45Z', 'bucket_order': 0, 'no_answer_followup': 0, 'change_log': [{'anon': 'no', 'uid': 'gd6v7134AUa', 'data': 'lsu5y3k0lul7ii', 'v': 'private', 'type': 'create', 'when': '2024-02-20T09:28:45Z'}], 'bucket_name': 'Pinned', 'history': [{'anon': 'no', 'uid': 'gd6v7134AUa', 'subject': 'Search for Teammates!', 'created': '2024-02-20T09:28:45Z', 'content': '<script type="text/javascript">PA.load("/dashboard/project_partners", null, function(data){ $(\'#\' + \'questionText\').html(data);});</script> #pin'}], 'type': 'note', 'anon_map': {}, 'tags': ['pin', 'student'], 'tag_good': [], 'unique_views': 1, 'children': [], 'tag_good_arr': [], 'anon_icons': True, 'id': 'lsu5y3jryuq7ig', 'config': {'feed_groups': 'instr_lsu5y35oppg7hs', 'is_default': 1}, 'status': 'private', 'drafts': None, 'request_instructor': 0, 'request_instructor_me': False, 'bookmarked': 0, 'num_favorite